In [17]:
import numpy as np
import pandas as pd
from keras.datasets import fashion_mnist
from sklearn.datasets import fetch_20newsgroups
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial.distance import squareform, pdist
from sklearn.feature_extraction.text import TfidfVectorizer
from math import sqrt
from sklearn import metrics
from sklearn.metrics import silhouette_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Functions for DBSCAN

In [12]:
def get_neighbors(i, df, euc_dist, eps):

    neighbors = []

    for j in range(len(df)):
        if i != j:
            d = euc_dist[i][j]
            if d > 0 and d <= eps:
                neighbors.append(j)

    return neighbors
  

In [13]:
def get_cluster_num(df, clusters):

    num = np.full(len(df), fill_value=-1)

    for i, cluster in enumerate(clusters):
        for c in cluster:
            num[c] = i+1
  
    return num

In [14]:
def get_purity(actual, cluster_lb, n):

    purity_df = pd.DataFrame({'actual':actual, 'cluster_lb':cluster_lb})
    purity_df = purity_df.groupby(['actual','cluster_lb'],as_index=False)['cluster_lb'].count().rename(columns={'cluster_lb':'count_cluster_lb'})
    purity_df = purity_df.sort_values(['actual','count_cluster_lb'],ascending=[True,False]).drop_duplicates(['actual'],keep='first')

    return round(purity_df['count_cluster_lb'].sum()/n, 2)

In [15]:
def get_gini(actual, cluster_lb, n):

    cluster_lb = np.array(cluster_lb).astype(int)
    actual = np.array(actual).astype(int)

    gini = 0

    for label in np.unique(actual):

        index = np.where(actual == label)[0]

        if len(index) > 0:
            p = (np.bincount(np.abs(cluster_lb[index]).astype(int))) / len(index)
            gini += len(index) / len(actual) * (1 - np.sum(p ** 2))
            
    return round(gini,2)

### Fashion MNIST

In [19]:
data = fashion_mnist.load_data()
(x_train, y_train), (x_test, y_test) = data
x_train = x_train.reshape((x_train.shape[0], 28*28)).astype('float32')

# Normalization: Scaling pixel values between 0 and 1

x_train = x_train / 255
x_test = x_test / 255


# Converting image pixels and label into train and test dataframe

xtrain = list()
ytrain = list()

for i in range(len(x_train)):
  xtrain.append(x_train[i])
  ytrain.append(y_train[i])

xtrain_df = pd.DataFrame({
    'Data': xtrain,
    'Label': ytrain
})


# Taking a sample of 500 images for each digit from train

fashion_df = xtrain_df.groupby('Label', group_keys=False).apply(pd.DataFrame.sample, n=100)


# Random shuffling dataframe

fashion_df = fashion_df.sample(frac=1)

4422102/4422102 [==============================] - 0s 0us/step


In [20]:
# Pre-compute euclidean distances

fashion_ed = [[0 for x in range(1000)] for y in range(1000)] 

for i in range(1000):
  for j in range(i//2):
    fashion_ed[i][j] = sqrt( np.dot(fashion_df.iloc[i,0], fashion_df.iloc[i,0]) - 2 * np.dot(fashion_df.iloc[i,0], fashion_df.iloc[j,0]) + np.dot(fashion_df.iloc[j,0], fashion_df.iloc[j,0]) )
    fashion_ed[j][i] = fashion_ed[i][j]


In [21]:
dbscan = []
min_eps = round(np.mean(fashion_ed),2) - 1
max_eps = round(np.mean(fashion_ed),2) + 0.1

for e in np.arange(min_eps, max_eps, 0.1, dtype=float):

    eps = e

    for ms in np.arange(3,11,1):
        
        min_samples = ms
        visited_samples = [False for i in range(len(fashion_df))]
        cluster_lb = [-1 for i in range(len(fashion_df))]
        neighbors = {}

        core = []
        non_core = []


        # Neighborhood Creation
        # Assigning points as core and non-core

        for i in range(len(fashion_df)):
            neighbors[i] = get_neighbors(i, fashion_df, fashion_ed, eps)
            if len(neighbors[i]) >= min_samples:
                core.append(i)
            else:
                non_core.append(i)


        # Iterate through samples and expand clusters from them if they have more neighbors than min_samples

        cn = 1

        for i in core:

            if visited_samples[i]:
                continue

            stack = [i]
            cluster_lb[i] = cn

            while stack:

                n = stack.pop()

                if visited_samples[n]:
                    continue

                visited_samples[n] = True

                if n in core:
                    for nn in neighbors[n]:
                        if cluster_lb[nn] == -1:
                            cluster_lb[nn] = cn
                    stack.extend(neighbors[n])
                else:
                    cluster_lb[n] = cn

            cn += 1

        res = {}
        res['eps'] = e
        res['min_samples'] = ms
        res['num_clusters'] = len(set(cluster_lb))
        res['cluster_lb'] = cluster_lb
        res['purity'] = get_purity(fashion_df['Label'], cluster_lb, 1000)
        res['gini'] = get_gini(fashion_df['Label'], cluster_lb, 1000)

        dbscan.append(res)


In [22]:
max_purity = 0
cst = 0

for i in range(len(dbscan)):
    if (dbscan[i]['purity'] > max_purity) and (dbscan[i]['num_clusters'] >= 8 and dbscan[i]['num_clusters'] <= 12):
        max_purity = dbscan[i]['purity']
        cst = i

In [23]:
print('Best Purity:', dbscan[cst]['purity'])
print('Best Gini:', dbscan[cst]['gini'])
print('No. of Clusters:', dbscan[cst]['num_clusters'])
print('Epsilon:', round(dbscan[cst]['eps'],2))
print('Min. no. of neighbors:', dbscan[cst]['min_samples'])

Max. Purity: 0.82
Max. Gini: 0.23
No. of Clusters: 8
Epsilon: 4.7
Min. no. of neighbors: 8


### 20NG

In [25]:
train_data = fetch_20newsgroups(subset='train')
xtrain_df = pd.DataFrame({
    'Data': train_data.data,
    'Label': train_data.target
})

# Taking a sample of 100 articles for each group for train

news_df = xtrain_df.groupby('Label', group_keys=False).apply(pd.DataFrame.sample, n=100)


# Random shuffling dataframe

news_df = news_df.sample(frac=1)

categories = train_data.target_names


# Vectorizing the articles

vectorizer = TfidfVectorizer()

vectors = vectorizer.fit_transform(news_df.Data)
vectors.shape

(2000, 47022)

In [26]:
# Pre-compute euclidean distances

news_eucd = metrics.pairwise.euclidean_distances(vectors)

In [35]:
dbscan = []
min_eps = round(np.mean(news_eucd),2) - 1
max_eps = round(np.mean(news_eucd),2) + 0.1

for e in np.arange(min_eps, max_eps, 0.1, dtype=float):

    eps = e

    for ms in np.arange(2,11,1):
        
        min_samples = ms
        visited_samples = [False for i in range(len(news_df))]
        cluster_lb = [-1 for i in range(len(news_df))]
        neighbors = {}

        core = []
        non_core = []


        # Neighborhood Creation
        # Assigning points as core and non-core

        for i in range(len(news_df)):
            neighbors[i] = get_neighbors(i, news_df, news_eucd, eps)
            if len(neighbors[i]) > min_samples:
                core.append(i)
            else:
                non_core.append(i)


        # Iterate through samples and expand clusters from them if they have more neighbors than min_samples

        cn = 1

        for i in core:

            if visited_samples[i]:
                continue

            stack = [i]
            cluster_lb[i] = cn

            while stack:

                n = stack.pop()

                if visited_samples[n]:
                    continue

                visited_samples[n] = True

                if n in core:
                    for nn in neighbors[n]:
                        if cluster_lb[nn] == -1:
                            cluster_lb[nn] = cn
                    stack.extend(neighbors[n])
                else:
                    cluster_lb[n] = cn

            cn += 1

        res = {}
        res['eps'] = e
        res['min_samples'] = ms
        res['num_clusters'] = len(set(cluster_lb))
        res['cluster_lb'] = cluster_lb
        res['purity'] = get_purity(news_df['Label'], cluster_lb, 2000)
        res['gini'] = get_gini(news_df['Label'], cluster_lb, 2000)

        dbscan.append(res)


In [38]:
max_purity = 0
cst = 0

for i in range(len(dbscan)):
    if (dbscan[i]['purity'] > max_purity) and (dbscan[i]['num_clusters'] >= 15 and dbscan[i]['num_clusters'] <= 20):
        max_purity = dbscan[i]['purity']
        cst = i

In [39]:
print('Best Purity:', dbscan[cst]['purity'])
print('Best Gini:', dbscan[cst]['gini'])
print('No. of Clusters:', dbscan[cst]['num_clusters'])
print('Epsilon:', round(dbscan[cst]['eps'],2))
print('Min. no. of neighbors:', dbscan[cst]['min_samples'])

Best Purity: 0.92
Best Gini: 0.08
No. of Clusters: 20
Epsilon: 1.08
Min. no. of neighbors: 2


### Household 


In [20]:
with open('/content/drive/MyDrive/UML/HW3/household_power_consumption.txt') as f:
    lines = f.readlines()

household_df = [line.strip().split(";") for line in lines]
household_df = pd.DataFrame(household_df[1:2001], columns=household_df[0])
household_df = household_df.drop(['Date', 'Time'], axis=1)
household_df = household_df.replace('?', np.nan).replace('', np.nan)
household_df = household_df.dropna().reset_index(drop=True)
household_df = household_df.astype(float)

household_df

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,3.666,0.528,235.68,15.8,0.0,1.0,17.0
...,...,...,...,...,...,...,...
1995,0.318,0.140,246.58,1.4,0.0,0.0,0.0
1996,0.312,0.138,245.93,1.4,0.0,0.0,0.0
1997,0.310,0.138,246.03,1.4,0.0,0.0,0.0
1998,0.308,0.138,245.98,1.4,0.0,0.0,0.0


In [21]:
# Pre-compute euclidean distances

house_eucd = metrics.pairwise.euclidean_distances(household_df)

In [23]:
dbscan = []
min_eps = round(np.mean(house_eucd),2) - 2
max_eps = round(np.mean(house_eucd),2) + 0.1

for e in np.arange(min_eps, max_eps, 0.2, dtype=float):

    eps = e

    for ms in np.arange(2,11,1):
        
        min_samples = ms
        visited_samples = [False for i in range(len(household_df))]
        cluster_lb = [-1 for i in range(len(household_df))]
        neighbors = {}

        core = []
        non_core = []


        # Neighborhood Creation
        # Assigning points as core and non-core

        for i in range(len(household_df)):
            neighbors[i] = get_neighbors(i, household_df, house_eucd, eps)
            if len(neighbors[i]) > min_samples:
                core.append(i)
            else:
                non_core.append(i)


        # Iterate through samples and expand clusters from them if they have more neighbors than min_samples

        cn = 1

        for i in core:

            if visited_samples[i]:
                continue

            stack = [i]
            cluster_lb[i] = cn

            while stack:

                n = stack.pop()

                if visited_samples[n]:
                    continue

                visited_samples[n] = True

                if n in core:
                    for nn in neighbors[n]:
                        if cluster_lb[nn] == -1:
                            cluster_lb[nn] = cn
                    stack.extend(neighbors[n])
                else:
                    cluster_lb[n] = cn

            cn += 1

        res = {}
        res['eps'] = e
        res['min_samples'] = ms
        res['num_clusters'] = len(set(cluster_lb))
        res['cluster_lb'] = cluster_lb
        res['silhouette_score'] = silhouette_score(house_eucd, cluster_lb)

        dbscan.append(res)


In [24]:
max_silhouette = 0
cst = 0

for i in range(len(dbscan)):
    if (dbscan[i]['silhouette_score'] > max_silhouette):
        max_silhouette = dbscan[i]['silhouette_score']
        cst = i

In [26]:
print('Best Silhouette Score:', round(dbscan[cst]['silhouette_score'],2))
print('No. of Clusters:', dbscan[cst]['num_clusters'])
print('Epsilon:', round(dbscan[cst]['eps'],2))
print('Min. no. of neighbors:', dbscan[cst]['min_samples'])

Best Silhouette Score: 0.62
No. of Clusters: 4
Epsilon: 15.08
Min. no. of neighbors: 4
